In [1]:
!pip install "transformers>=4.41.0,<5.0.0" datasets peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.3 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
notebook_login()


In [3]:
from datasets import load_dataset

# Load full dataset
ds = load_dataset("bitext/Bitext-retail-ecommerce-llm-chatbot-training-dataset")

# Select 2000 samples for Colab
small_train_ds = ds["train"].shuffle(seed=42).select(range(2000))

# Split into train/test
train_ds = small_train_ds.select(range(1500))
eval_ds = small_train_ds.select(range(1500, 2000))

print(train_ds[0])


README.md: 0.00B [00:00, ?B/s]

bitext-retail-ecommerce-llm-chatbot-trai(…):   0%|          | 0.00/42.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44884 [00:00<?, ? examples/s]

{'instruction': 'I have to report a fucking wrong product, how do I do it?', 'intent': 'wrong_item', 'category': 'DELIVERY', 'tags': 'BCILW', 'response': 'I sincerely apologize for any frustration caused by receiving the incorrect product in your order. I understand how important it is to resolve this issue for you. Let me guide you on how to report the wrong item:\n\n1. Start by logging into your account on our website or app.\n2. Navigate to your order history or order details page.\n3. Locate the order containing the wrong item.\n4. Click on the "Report a Problem" or "Contact Us" option related to the order.\n5. Provide a brief explanation of the issue and mention that you received the wrong product.\n6. Include any relevant details such as the order number, product name, and a description of the incorrect item received.\n7. Submit the report or message to initiate the resolution process.\n\nOur customer support team will review your report and assist you promptly in resolving the i

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-2-7b-hf"   # 7B model (largest you can *maybe* handle with LoRA)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,    # quantization to save GPU memory
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [5]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # optimize attention projections
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, config)


In [6]:
def tokenize_function(examples):
    # Use 'instruction' as input and 'response' as output
    inputs = [f"Customer: {q}\nAgent:" for q in examples["instruction"]]
    targets = [str(a) for a in examples["response"]]

    model_inputs = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    labels = tokenizer(
        targets,
        truncation=True,
        padding="max_length",
        max_length=256
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

train_tokenized = train_ds.map(tokenize_function, batched=True)
eval_tokenized = eval_ds.map(tokenize_function, batched=True)


Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="steps", # Corrected argument name
    save_strategy="steps",
    save_steps=100,
    eval_steps=50,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=eval_tokenized,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipython-input-1924542933.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss,Validation Loss
50,5.085300,4.839148
100,4.248700,4.392307
150,4.388900,4.308845


TrainOutput(global_step=188, training_loss=5.776371712380267, metrics={'train_runtime': 1192.6789, 'train_samples_per_second': 1.258, 'train_steps_per_second': 0.158, 'total_flos': 1.5242647044096e+16, 'train_loss': 5.776371712380267, 'epoch': 1.0})

In [8]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 4.27944278717041, 'eval_runtime': 96.2393, 'eval_samples_per_second': 5.195, 'eval_steps_per_second': 2.598, 'epoch': 1.0}


In [ ]:
def chat_with_model(prompt):
    inputs = tokenizer(f"Customer: {prompt}\nAgent:", return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(chat_with_model("I want to buy a new laptop"))
print(chat_with_model("Do you have discounts on shoes?"))


In [ ]:
trainer.save_model("./llama2-finetuned")

# Reload later:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("./llama2-finetuned")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def chat_loop():
    print("🤖 Chatbot ready! (type 'exit' to quit)\n")
    history = ""

    while True:
        user_input = input("Customer: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Agent: Okay, bye! 👋")
            break

        # Keep conversation history
        history += f"\nCustomer: {user_input}\nAgent:"

        inputs = tokenizer(history, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract only the new agent part
        agent_reply = response.split("Agent:")[-1].strip()
        print(f"Agent: {agent_reply}")

        # Append response to history for context
        history += f" {agent_reply}"

# Run chatbot
chat_loop()
